In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
archivos = [
  'Benedetti.txt',
  'Neruda.txt',
]
print (archivos)
textos = []
etiquetas = []

['Benedetti.txt', 'Neruda.txt']


In [3]:
for etiqueta, f in enumerate(archivos):
    print(f"{f} Corresponde a {etiqueta}")    
    with open(f, 'r', encoding='utf-8') as archivo:
        for line in archivo:
            print(line)
            line = line.rstrip().lower()
            print(line)
            if line:
                # eliminar puntuación
                line = line.translate(str.maketrans('', '', string.punctuation))
                textos.append(line)
                etiquetas.append(etiqueta)
            print(line)
train_text, test_text, Ytrain, Ytest = train_test_split(textos, etiquetas, test_size=0.1, random_state=42)
len(Ytrain)
len(Ytest)

Benedetti.txt Corresponde a 0
Poemas varios

poemas varios
poemas varios
Mario Benedetti

mario benedetti
mario benedetti




La buena tiniebla

la buena tiniebla
la buena tiniebla
Una mujer desnuda y en lo oscuro

una mujer desnuda y en lo oscuro
una mujer desnuda y en lo oscuro
genera un resplandor que da confianza

genera un resplandor que da confianza
genera un resplandor que da confianza
de modo que si sobreviene

de modo que si sobreviene
de modo que si sobreviene
un apagón o un desconsuelo

un apagón o un desconsuelo
un apagón o un desconsuelo
es conveniente y hasta imprescindible

es conveniente y hasta imprescindible
es conveniente y hasta imprescindible
tener a mano una mujer desnuda

tener a mano una mujer desnuda
tener a mano una mujer desnuda
entonces las paredes se acuarelan

entonces las paredes se acuarelan
entonces las paredes se acuarelan
el cielo raso se convierte en cielo

el cielo raso se convierte en cielo
el cielo raso se convierte en cielo
las telarañas vibran e

200

In [4]:
"""
"<unk>" es una convención que se utiliza a menudo en el procesamiento de lenguaje natural para representar palabras 
desconocidas o fuera del vocabulario. En este caso, se está asignando el índice 0 a esta palabra especial.
"""

'\n"<unk>" es una convención que se utiliza a menudo en el procesamiento de lenguaje natural para representar palabras \ndesconocidas o fuera del vocabulario. En este caso, se está asignando el índice 0 a esta palabra especial.\n'

In [5]:
indice = 1
indicepalabras = {'<unk>': 0}
for texto in train_text:
    tokens = texto.split()
    for token in tokens:
        if token not in indicepalabras:
            indicepalabras[token] = indice
            indice += 1

train_text_int = []

test_text_int = []

In [6]:
for texto in train_text:
    tokens = texto.split()
    linea_entero = [indicepalabras[token] for token in tokens]
    train_text_int.append(linea_entero)

for texto in test_text:
    tokens = texto.split()
    line_as_int = [indicepalabras.get(token, 0) for token in tokens]
    test_text_int.append(line_as_int)

In [7]:
V = len(indicepalabras)
A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [18]:
def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            # estamos en la primera palabra de la secuencia
            if last_idx is None:
                pi[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

In [9]:
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

count0 = sum(y == 0 for y in Ytrain)   
count1 = sum(y == 1 for y in Ytrain)   
total = len(Ytrain)   
p0 = count0 / total   
p1 = count1 / total   
logp0 = np.log(p0)   
logp1 = np.log(p1) 

In [12]:
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) # número de clases
        
    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                # Es el primer token en la secuencia
                logprob += logpi[idx]
            else:
                # Calcula la probabilidad de transición de la palabra anterior a la actual
                logprob += logA[last_idx, idx]
            
            # Actualiza last_idx para la próxima iteración
            last_idx = idx
        
        return logprob     
    
    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            # Calcula los logaritmos de las probabilidades posteriores para cada clase
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                          for c in range(self.K)]
            # Elige la clase con la mayor probabilidad posterior como la predicción
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions

In [15]:
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)*100}%")

Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)*100}%")

Train acc: 99.2749581706637%
Test acc: 83.0%


In [21]:
texto = "Cuerpo de piel, de musgo, de leche ávida y firme."
tokens = texto.split()
input_ = [indicepalabras.get(token, 0) for token in tokens]
pred = clf.predict([input_])[0]
print(f"El texto pertenece a la clase {pred}")

El texto pertenece a la clase 1.0
